In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

Implementation of Table Creation:
1. Entities Table (ID, comp_name, yahoo_ticker, refinitiv_ticker)
2. Exchange Table (ID, name, country)
3. Asset Classes Table (ID, name)
2. Securities Table 
    - ID: Unique Primary Key
    - entity_ID: Reference to Entities Table
    - isin: / 
    - exchange_ID: Reference to Exchange Table
    - asset_classes: Reference to Asset Classes Table
3. Securities_time_series Table

Here, I would like to establish connection and create demo query

In [ ]:
from sqlalchemy import create_engine

# Create connection string 
host = "financial-database.mysql.database.azure.com"
database = "currency_data" 
username = "cuqts_datateam"
password = ######
port = 3306

connection_str = f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}'

engine = create_engine(connection_str, 
                       connect_args={"ssl": {"ssl_ca": "DigiCertGlobalRootCA.crt.pem"}})
connection = engine.connect()

In [ ]:
query = """SELECT * FROM securities;"""

# If the query is using select then will have result
result = connection.execute(query)


1. Entities Table Creation

In [ ]:
data = pd.read_csv('SP500.csv').drop(columns=['date'])

column_lst = ['ID', 'comp_name', 'yahoo_ticker', 'refinitiv_ticker']
entity_table = pd.DataFrame(columns = column_lst)
entity_table['ID'] = [i for i in range(len(data.columns))]
entity_table['comp_name'] = data.columns
entity_table['yahoo_ticker'] = data.columns
entity_table['refinitiv_ticker'] = data.columns

In [ ]:
entity_table.to_sql('entities', con=engine, if_exists='append', index=False)

2. Exchange Table Creation

In [ ]:
data = {
    'ID': [1],
    'Name': ['spy500'],
    'Country': ['USA']
}

excxhange_df = pd.DataFrame(data)

In [ ]:
excxhange_df.to_sql('exchanges', con=engine, if_exists='append', index=False)

3. Asset Class Table Creation

In [ ]:
data = {
    'ID': [1, 2, 3, 4, 5],
    'Name': ['Equity', 'Fixed Income', 'Commodity', 'Currency', 'Real Estate']
}

asset_df = pd.DataFrame(data)

In [ ]:
asset_df.to_sql('asset_classes', con=engine, if_exists='append', index=False)

4. Securities Table Creation

In [34]:
data = pd.read_csv('SP500.csv').drop(columns=['date'])

column_lst = ['ID', 'entity_ID', 'exchange_ID', 'asset_class_ID', 'isin']

entities_table = pd.read_sql_table('entities', con=engine)
entity_ids = entities_table['ID'].tolist()
security_table = pd.DataFrame(columns = column_lst)
security_table['ID'] = [i for i in range(len(data.columns))]
security_table['entity_ID'] = entity_ids
security_table['exchange_ID'] = 1
security_table['asset_class_ID'] = 1
security_table['isin'] = ''

security_table.to_sql('securities', con=engine, if_exists='append', index=False)

5. Securities Time Series Table

In [72]:
for idx, symbol in enumerate(data.columns[1:]):
    ticker = yf.Ticker('AAPL')  # Replace "AAPL" with your desired stock symbol

    # Get historical stock data for Open, High, Low, Close, and Volume
    history = ticker.history(period="5y")[['Open', 'High', 'Low', 'Close', 'Volume']]  # Replace "1mo" with the desired period

    new_data = history
    # Check for duplicate entries
    # existing_dates = set(history.index)  # Get the existing dates in the table
    # new_data = history[~history.index.isin(existing_dates)]  # Exclude the existing dates from the new data
    # print(new_data)
    # Prepare the new data for insertion
    new_data['datetime'] = new_data.index
    new_data['interval'] = '1D'
    new_data['amount'] = 0
    new_data['security_ID'] = idx + 1

    new_data.to_sql('securities_time_series', con = engine, if_exists='append', index=False)

# Insert the new data into the table
# Write your code for database insertion here

AAL
                                 Open        High         Low       Close  \
Date                                                                        
2018-09-20 00:00:00-04:00   52.732812   53.221255   52.471827   52.682529   
2018-09-21 00:00:00-04:00   52.862121   53.000993   52.026497   52.115089   
2018-09-24 00:00:00-04:00   51.913955   52.977036   51.868462   52.864502   
2018-09-25 00:00:00-04:00   52.615500   53.350563   52.603528   53.199718   
2018-09-26 00:00:00-04:00   52.914785   53.573227   52.617886   52.775913   
...                               ...         ...         ...         ...   
2023-09-14 00:00:00-04:00  174.000000  176.100006  173.580002  175.740005   
2023-09-15 00:00:00-04:00  176.479996  176.500000  173.820007  175.009995   
2023-09-18 00:00:00-04:00  176.479996  179.380005  176.169998  177.970001   
2023-09-19 00:00:00-04:00  177.520004  179.630005  177.130005  179.070007   
2023-09-20 00:00:00-04:00  179.259995  179.679993  177.690308  177.69999